## Stage 03: Binomial Tree Model
- Maturity (T): 1-5 trading days
- Strikes (K): based on today's close price (adj_close), with offsets: {-$2, -$1, $0, +$1, +$2}
- Outputs: Call & Put prices for each (T, K) pair
- Models: Binomial Tree

In [2]:
# project path setup
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().parent
SRC_PATH = PROJECT_ROOT / "src"

if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

### Compute Binomial Tree Cox-Ross-Rubinstein(CRR) Model

In [26]:
# load modules
from pathlib import Path
import pandas as pd

from options_valuation.utils.strikes import make_strike_grid
from options_valuation.utils.market_info import get_latest_spot_info
from options_valuation.pricing.binomial import price_crr_grid
from options_valuation.data.rates import get_risk_free_rate_yf

# Load vol table
vol_path = PROJECT_ROOT / "data" / "processed" / "volatility_inputs.csv"
vol_all = pd.read_csv(vol_path)
vol_all["horizon_days"] = vol_all["horizon_days"].astype(int)

# Load the stock info that is working on
spot = get_latest_spot_info(PROJECT_ROOT, ticker=None)  # infer ticker automatically
ticker = spot.ticker
S0 = spot.S0
asof_date = spot.asof_date

print(f"{ticker}: S0={S0:.2f} as of {asof_date} ({spot.price_col})")

# Load the risk-free rate used
rf = get_risk_free_rate_yf(symbol="^IRX", lookback_days=10, fallback_r=0.05)
r = rf.r_annual

print(f"Risk-free r={r:.4%} as of {rf.asof_date} from {rf.symbol} ({rf.source_col})")
print("Note:", rf.note)

# Build strike grid
K_grid = make_strike_grid(S0)

# Price both models
price_table = price_crr_grid(
    ticker=ticker,
    S0=S0,
    r_annual=r,
    K_grid=K_grid,
    vol_table=vol_all,
    models=("garch", "hist_rolling"),
    horizons=range(1, 6),
)

display(price_table)

# Save output
out_path = PROJECT_ROOT / "data" / "processed" / f"{ticker}_crr_option_prices.csv"
out_path.parent.mkdir(parents=True, exist_ok=True)
price_table.to_csv(out_path, index=False)
print("Saved:", out_path)

AAPL: S0=260.33 as of 2026-01-07 (adj_close)
Risk-free r=3.5150% as of 2026-01-07 from ^IRX (Close)
Note: Converted percent quote to decimal


,ticker,asof_S0,K,T_days,vol_model,sigma_h,sigma_ann,r_annual,call_CRR,put_CRR
0,AAPL,260.329987,258.33,1,garch,0.014742,0.234030,0.03515,2.938886,0.902869
1,AAPL,260.329987,259.33,1,garch,0.014742,0.234030,0.03515,2.437911,1.401755
2,AAPL,260.329987,260.33,1,garch,0.014742,0.234030,0.03515,1.936936,1.900640
3,AAPL,260.329987,261.33,1,garch,0.014742,0.234030,0.03515,1.435960,2.399525
4,AAPL,260.329987,262.33,1,garch,0.014742,0.234030,0.03515,0.934985,2.898410
5,AAPL,260.329987,258.33,2,garch,0.021167,0.237599,0.03515,3.485793,1.413751
6,AAPL,260.329987,259.33,2,garch,0.021167,0.237599,0.03515,2.735085,1.662764
7,AAPL,260.329987,260.33,2,garch,0.021167,0.237599,0.03515,1.984384,1.911784
8,AAPL,260.329987,261.33,2,garch,0.021167,0.237599,0.03515,1.733535,2.660656
9,AAPL,260.329987,262.33,2,garch,0.021167,0.237599,0.03515,1.482687,3.409528


Saved: /Users/allenpilipala/BSM_model/data/processed/AAPL_crr_option_prices.csv
